# Predict if there is a leakge

We select a random hour h in the dataset. We take the previous 384 hours (h-384, ... h-1), we make the prediction for h, h+1, ..., h+12. We compute the TP, FP, precision and we compare it to the real values in the dataframe

In [5]:
import numpy as np
import pandas as pd
import torch
from einops import rearrange
from uni2ts.model.moirai import MoiraiForecast, MoiraiModule

df_data = pd.read_csv("../../data/ES=F.csv", parse_dates=True, index_col=0)
random_index = 4560
ground_truth_rows = df_data.iloc[random_index: random_index + 12]
context_rows = df_data.iloc[random_index - 384: random_index]
initial_row = df_data.iloc[random_index - 1]

model = MoiraiForecast(
    module=MoiraiModule.from_pretrained(
        f"Salesforce/moirai-1.1-R-base"
    ),
    prediction_length=12,
    context_length=384,
    patch_size=32,
    num_samples=100,
    target_dim=1,
    feat_dynamic_real_dim=0,
    past_feat_dynamic_real_dim=0,
)

context_close_denoised_std = context_rows["Close_denoised_standardized"].to_numpy()

past_target = rearrange(
    torch.as_tensor(context_close_denoised_std, dtype=torch.float32), "t -> 1 t 1"
)

past_observed_target = torch.ones_like(past_target, dtype=torch.bool)
past_is_pad = torch.zeros_like(past_target, dtype=torch.bool).squeeze(-1)

# Make the forecast
forecast = model(
    past_target=past_target,
    past_observed_target=past_observed_target,
    past_is_pad=past_is_pad,
)

results = np.round(np.median(forecast[0], axis=0), decimals=4)
destandardized_results = [result * df_data["Close_denoised"].std() + df_data["Close_denoised"].mean() for result in results]
base_price = initial_row["Close"]

TP = [0 for _ in range(12)]
FP = [0 for _ in range(12)]
FN = [0 for _ in range(12)]
TN = [0 for _ in range(12)]

for i, (gt, pred) in enumerate(zip(ground_truth_rows["Close"], destandardized_results)):
    sign_gt = np.sign(gt - base_price) # if up then 1, if down then -1
    sign_pred = np.sign(pred - base_price)

    if sign_gt == sign_pred and sign_gt == 1:
        TP[i] += 1
    elif sign_gt == sign_pred and sign_gt == -1:
        TN[i] += 1
    elif sign_gt != sign_pred and sign_gt == 1:
        FN[i] += 1
    elif sign_gt != sign_pred and sign_gt == -1:
        FP[i] += 1

print("Predictions precision: ")
print(TP, FP, FN, TN)

df_moirai = pd.read_csv("../../analysis/future_data/es_future_final_moirai_updated.csv", parse_dates=True, index_col=0)
init_row = df_moirai.iloc[random_index - 1]
print("Moirai precision: ")
print(init_row["TP"], init_row["FP"], init_row["FN"], init_row["TN"])

Predictions precision: 
[0, 0, 0, 0, 1, 0, 1, 1, 1, 1, 1, 1] [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0] [0, 1, 1, 1, 0, 1, 0, 0, 0, 0, 0, 0] [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
Moirai precision: 
[0, 0, 1, 1, 1, 1, 1, 1, 1, 1, 0, 1] [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0] [0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0] [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]
